In [10]:
import pandas as pd
import os, string
import re
from keras.preprocessing.text import Tokenizer
from gensim.models.fasttext import FastText
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer
import spacy
from spacy.tokenizer import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction import text
import nltk

# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
pd.set_option('display.max_columns', 500)

https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object

#### Read data

In [25]:
df = pd.read_csv('tweets.csv')

In [29]:
df=df.drop(columns='Unnamed: 0')

#### EDA

In [65]:
df.dtypes

created_at                    object
display_text_range            object
entities                      object
extended_entities             object
favorite_count                 int64
favorited                       bool
full_text                     object
id                             int64
id_str                         int64
in_reply_to_screen_name       object
in_reply_to_status_id        float64
in_reply_to_status_id_str    float64
in_reply_to_user_id          float64
in_reply_to_user_id_str      float64
is_quote_status                 bool
lang                          object
place                         object
possibly_sensitive            object
quoted_status                 object
quoted_status_id             float64
quoted_status_id_str         float64
quoted_status_permalink       object
retweet_count                  int64
retweeted                       bool
retweeted_status              object
source                        object
truncated                       bool
u

In [69]:
def print_na_values():
    all_cols = df.columns
    print("Total shape: ", df.shape)
    for col in all_cols:
        if(len(df[df[col].isna()][col]) != 0):
            print(f"Na values for col {col}: {len(df[df[col].isna()][col])}")

In [80]:
def print_null_values():
    all_cols = df.columns
    print("Total shape: ", df.shape)
    for col in all_cols:
        if(len(df[df[col].isnull()][col]) != 0):
            print(f"Null values for col {col}: {len(df[df[col].isna()][col])}")

In [68]:
print_na_values()

Total shape:  (6159, 28)
Na values for col extended_entities: 5370
Na values for col in_reply_to_screen_name: 2514
Na values for col in_reply_to_status_id: 2540
Na values for col in_reply_to_status_id_str: 2540
Na values for col in_reply_to_user_id: 2514
Na values for col in_reply_to_user_id_str: 2514
Na values for col place: 6156
Na values for col possibly_sensitive: 5161
Na values for col quoted_status: 6041
Na values for col quoted_status_id: 6002
Na values for col quoted_status_id_str: 6002
Na values for col quoted_status_permalink: 6002
Na values for col retweeted_status: 5492


In [81]:
print_null_values()

Total shape:  (6159, 28)
Null values for col extended_entities: 5370
Null values for col in_reply_to_screen_name: 2514
Null values for col in_reply_to_status_id: 2540
Null values for col in_reply_to_status_id_str: 2540
Null values for col in_reply_to_user_id: 2514
Null values for col in_reply_to_user_id_str: 2514
Null values for col place: 6156
Null values for col possibly_sensitive: 5161
Null values for col quoted_status: 6041
Null values for col quoted_status_id: 6002
Null values for col quoted_status_id_str: 6002
Null values for col quoted_status_permalink: 6002
Null values for col retweeted_status: 5492


In [66]:
df.describe()

,favorite_count,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,quoted_status_id,quoted_status_id_str,retweet_count
count,6159.000000,6.159000e+03,6.159000e+03,3.619000e+03,3.619000e+03,3.645000e+03,3.645000e+03,1.570000e+02,1.570000e+02,6159.000000
mean,2061.684202,1.010494e+18,1.010494e+18,1.073790e+18,1.073790e+18,2.625706e+17,2.625706e+17,9.742269e+17,9.742269e+17,992.399578
std,14008.929534,1.692469e+17,1.692469e+17,1.023473e+17,1.023473e+17,4.185165e+17,4.185165e+17,1.526749e+17,1.526749e+17,5918.925656
min,0.000000,5.568955e+17,5.568955e+17,3.768168e+09,3.768168e+09,1.332121e+06,1.332121e+06,6.105016e+17,6.105016e+17,0.000000
25%,1.000000,9.894714e+17,9.894714e+17,1.014719e+18,1.014719e+18,3.336645e+08,3.336645e+08,8.877338e+17,8.877338e+17,0.000000
50%,7.000000,1.040752e+18,1.040752e+18,1.080976e+18,1.080976e+18,1.413627e+09,1.413627e+09,9.903306e+17,9.903306e+17,0.000000
75%,83.000000,1.125954e+18,1.125954e+18,1.145731e+18,1.145731e+18,7.326784e+17,7.326784e+17,1.058449e+18,1.058449e+18,15.000000
max,462885.000000,1.222370e+18,1.222370e+18,1.222370e+18,1.222370e+18,1.219086e+18,1.219086e+18,1.216535e+18,1.216535e+18,145351.000000


In [82]:
df.head()

,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,retweeted_status,source,truncated,user
0,Wed Jan 29 01:47:13 +0000 2020,"[11, 79]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,4,False,@Spothebro yes actually but it wasn’t sexy. mo...,1222335355543785472,1222335355543785472,Spothebro,1.222335e+18,1.222335e+18,1.788721e+08,1.788721e+08,False,en,NaN,NaN,NaN,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 2492132730, 'id_str': '2492132730', 'na..."
1,Tue Jan 28 18:38:44 +0000 2020,"[10, 53]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,451,False,@blaizun1 12 year old danya was absolutely dev...,1222227523150344193,1222227523150344193,blaizun1,1.222227e+18,1.222227e+18,1.164471e+18,1.164471e+18,False,en,NaN,NaN,NaN,NaN,NaN,NaN,2,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 2492132730, 'id_str': '2492132730', 'na..."
2,Tue Jan 28 18:20:25 +0000 2020,"[13, 35]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1222222908581265408, 'id_str...",454,False,@thats_messy i stole it from myself https://t....,1222222914583498752,1222222914583498752,thats_messy,1.222223e+18,1.222223e+18,1.102349e+18,1.102349e+18,False,en,NaN,False,NaN,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 2492132730, 'id_str': '2492132730', 'na..."
3,Tue Jan 28 18:14:47 +0000 2020,"[0, 246]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,34480,False,"one time in middle school this kid was like ""u...",1222221497390944256,1222221497390944256,NaN,NaN,NaN,NaN,NaN,False,en,NaN,NaN,NaN,NaN,NaN,NaN,1066,False,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,"{'id': 2492132730, 'id_str': '2492132730', 'na..."
4,Tue Jan 28 02:39:15 +0000 2020,"[13, 33]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,1,False,@KevinWicks2 peter isn’t cute man,1221986062026887168,1221986062026887168,KevinWicks2,1.221986e+18,1.221986e+18,1.538617e+09,1.538617e+09,False,en,NaN,NaN,NaN,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 2492132730, 'id_str': '2492132730', 'na..."


In [60]:
# first preselection based on description:
not_to_use = ["contributors", "coordinates", "geo", "id","id_str","in_reply_to_status_id","in_reply_to_status_id_str","in_reply_to_user_id","in_reply_to_user_id_str","quoted_status_id","quoted_status_id_str"]
df = df.drop(columns=to_remove)


In [ ]:
# We have as many ID as rows we can drop id column since we won't use it to scrape other tweets or see responses
df= df.drop()

#### Pre-process

Create 'joke' column and remove unnecessary columns:

In [33]:
df.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,retweeted_status,source,truncated,user
0,NaN,NaN,Wed Jan 29 01:47:13 +0000 2020,"[11, 79]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,4,False,@Spothebro yes actually but it wasn’t sexy. mo...,NaN,1222335355543785472,1222335355543785472,Spothebro,1.222335e+18,1.222335e+18,1.788721e+08,1.788721e+08,False,en,NaN,NaN,NaN,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 2492132730, 'id_str': '2492132730', 'na..."
1,NaN,NaN,Tue Jan 28 18:38:44 +0000 2020,"[10, 53]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,451,False,@blaizun1 12 year old danya was absolutely dev...,NaN,1222227523150344193,1222227523150344193,blaizun1,1.222227e+18,1.222227e+18,1.164471e+18,1.164471e+18,False,en,NaN,NaN,NaN,NaN,NaN,NaN,2,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 2492132730, 'id_str': '2492132730', 'na..."
2,NaN,NaN,Tue Jan 28 18:20:25 +0000 2020,"[13, 35]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1222222908581265408, 'id_str...",454,False,@thats_messy i stole it from myself https://t....,NaN,1222222914583498752,1222222914583498752,thats_messy,1.222223e+18,1.222223e+18,1.102349e+18,1.102349e+18,False,en,NaN,False,NaN,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 2492132730, 'id_str': '2492132730', 'na..."
3,NaN,NaN,Tue Jan 28 18:14:47 +0000 2020,"[0, 246]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,34480,False,"one time in middle school this kid was like ""u...",NaN,1222221497390944256,1222221497390944256,NaN,NaN,NaN,NaN,NaN,False,en,NaN,NaN,NaN,NaN,NaN,NaN,1066,False,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,"{'id': 2492132730, 'id_str': '2492132730', 'na..."
4,NaN,NaN,Tue Jan 28 02:39:15 +0000 2020,"[13, 33]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,1,False,@KevinWicks2 peter isn’t cute man,NaN,1221986062026887168,1221986062026887168,KevinWicks2,1.221986e+18,1.221986e+18,1.538617e+09,1.538617e+09,False,en,NaN,NaN,NaN,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 2492132730, 'id_str': '2492132730', 'na..."


In [8]:
df['joke'] = df['title'] +'. '+ df['body']

In [9]:
df = df.loc[:,['joke','score']]

Pre-process text:
- Remove punctuation
- Replace escape characters
- Remove extra spaces
- Remove single characters
- Remove prefixed 'b'
- Lowercase all characters
- Lemmatization

In [10]:
def replace_non_eng_punct(txt):
    return re.sub(r'/[^a-zA-Z0-9\s,.?!]/','*',txt).strip()

In [11]:
# def remove_url(txt):
#     return re.sub(r'https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}','',txt)

In [12]:
def replace_escape(txt):
    updated_txt = re.sub(r'\n|\t|&amp;',' ',txt)
    return updated_txt.strip()

In [13]:
def remove_multi_spaces(txt):
    return re.sub(' +', ' ',txt)

In [14]:
import re
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

def preprocess_text(document):
#         # Remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

#         # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

#         # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()

        # Lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in en_stop]
        tokens = [word for word in tokens if len(word) > 3]

        preprocessed_text = ' '.join(tokens)

        return preprocessed_text

In [15]:
df['joke'] = df['joke'].apply(replace_non_eng_punct).apply(remove_multi_spaces)

In [16]:
df['joke_preprocessed'] = df['joke'].apply(preprocess_text)

Sample before/after preprocessing:

In [17]:
df['joke'][0]

'I hate how you cant even say black paint anymore. Now I have to say "Leroy can you please paint the fence?"'

In [18]:
df['joke_preprocessed'][0]

'hate cant even black paint anymore leroy please paint fence'

Create joke tokens list:

In [19]:
def split_punct(txt):
    return re.split(r'(\W)',txt)

In [20]:
df['joke_tokens'] = df['joke_preprocessed'].apply(split_punct)

In [21]:
def remove_spaces(lst):
    return [x for x in lst if (x != ' ') and (x != '')]

In [22]:
df['joke_tokens'] = df['joke_tokens'].apply(remove_spaces)

#### Feature engineering

Train/test split:

In [31]:
target_col = 'score'
tokens_col = 'joke_preprocessed'
X = df[tokens_col].to_numpy()
y = df[target_col].to_numpy()

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, \
                                                    random_state=42)

Feature scaling:

In [33]:
scaler = MinMaxScaler()
scaler.fit(y_train.reshape(-1,1))
y_train = scaler.transform(y_train.reshape(-1,1)).reshape(-1,)
y_test = scaler.transform(y_test.reshape(-1,1)).reshape(-1,)

Generate vocabulary:

In [35]:
cnt = Counter()
for s in df['joke_tokens']:
    for w in s:
        cnt[w] += 1

In [36]:
vocab2 = cnt.most_common(1000)
vocab2 = [i[0] for i in vocab2]
# vocab_id = defaultdict(int)
# for ind,w in enumerate(vocab):
#     vocab_id[w[0]] = ind+2

Generate TFIDF/CountVectorizer features:

In [37]:
concat_filter_vec2 = np.vectorize(lambda x: ' '.join([i for i in x.split(' ') if i in vocab2]))

In [38]:
X_train = concat_filter_vec2(X_train)
X_test = concat_filter_vec2(X_test)

In [39]:
X_train[6]

'scientist came inside fuck dude hell away'

In [40]:
vectorizer_tf = TfidfVectorizer()
X_train_tf = vectorizer_tf.fit_transform(X_train).toarray()
X_test_tf = vectorizer_tf.transform(X_test).toarray()

In [41]:
c = CountVectorizer()
X_train_count = c.fit_transform(X_train).toarray()
X_test_count = c.fit_transform(X_test).toarray()

In [42]:
X_train_tf

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.33728993, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [43]:
X_train_count.shape

(106393, 1000)

In [44]:
from keras.models import Model, Sequential
from keras.layers import Activation, Dense, LSTM, Embedding, TimeDistributed, recurrent
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras import regularizers

#### Modeling

Train Neural network

In [45]:
# Neural network
model = Sequential()
model.add(Dense(128, input_dim=1000, activation='relu',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
model.add(Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
model.add(Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
model.add(Dense(1, activation='relu',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))

In [46]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               128128    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 161,281
Trainable params: 161,281
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mse'])

In [48]:
history = model.fit(X_train_tf, y_train, epochs=20, batch_size=200,validation_split = 0.2)




Train on 85114 samples, validate on 21279 samples
Epoch 1/20





85114/85114 [==============================] - 5s 54us/step - loss: 4.4947 - mean_squared_error: 5.1584e-04 - val_loss: 4.0598 - val_mean_squared_error: 6.4839e-04
Epoch 2/20
85114/85114 [==============================] - 4s 42us/step - loss: 3.7326 - mean_squared_error: 5.1399e-04 - val_loss: 3.4221 - val_mean_squared_error: 6.4839e-04
Epoch 3/20
85114/85114 [==============================] - 3s 38us/step - loss: 3.1471 - mean_squared_error: 5.1399e-04 - val_loss: 2.8858 - val_mean_squared_error: 6.4839e-04
Epoch 4/20
85114/85114 [==============================] - 3s 40us/step - loss: 2.6539 - mean_squared_error: 5.1399e-04 - val_loss: 2.4336 - val_mean_squared_error: 6.4839e-04
Epoch 5/20
85114/85114 [==============================] - 4s 43us/step - loss: 2.2381 - mean_squared_error: 5.1399e-04 - val_loss: 2.0524 - val_mean_squared_error: 6.4839e-04
Epoch 6/20
85114/85114 [==============================] - 3s 38us/s

In [49]:
y_pred = model.predict(X_test_tf)

In [54]:
y_pred.sum()

0.0

In [52]:
y_test

array([1.89592993e-03, 0.00000000e+00, 2.30808861e-03, ...,
       0.00000000e+00, 2.26687275e-04, 2.06079341e-05])